In [12]:
from protein_configuration import distance_residue, distance_cutoff
import MDAnalysis
from MDAnalysis.analysis import distances
import itertools
from numpy.lib.function_base import average
import pandas as pd
import dask
import dask.multiprocessing
from dask.distributed import Client
dask.config.set(scheduler='processes')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:45063 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 62.75 GiB


In [8]:
#plainMD_directory = '/home/emanuele/TTR/greta_cutoff_55_ex_2/epsilon_0275_latestdihedrals_newljr_harp0/monomer_test/plain_MD'
plainMD_directory = '/home/emanuele/ABeta/markov'
#reference_plainMD_structure = f'{plainMD_directory}/monomer_analysis.gro'
#reference_plainMD_trajectory = f'{plainMD_directory}/monomer_analysis.xtc'
reference_plainMD_structure = f'{plainMD_directory}/reduced-noh.gro'
reference_plainMD_trajectory = f'{plainMD_directory}/reduced-noh.xtc'

random_coil_directory = '/home/emanuele/ABeta/random_coil/monomer_test/native_278K'
reference_random_coil_structure = f'{random_coil_directory}/box_abeta_greta.gro'
reference_random_coil_trajectory = f'{random_coil_directory}/prod_abeta_greta.xtc'

In [9]:
def make_pairs(atomgroup, frame_index, pairs_ai, pairs_aj):

    framed_peptide = atomgroup.universe.trajectory[frame_index]
    self_distance = distances.self_distance_array(framed_peptide.positions)
    monomer_pairs_df = pd.DataFrame(columns=['ai', 'aj','ai_name', 'aj_name', 'ai_resnum', 'aj_resnum', 'distances'])
    monomer_pairs_df['ai'] = pairs_ai
    monomer_pairs_df['aj'] = pairs_aj
    monomer_pairs_df['distances'] = self_distance
    monomer_pairs_df = monomer_pairs_df[monomer_pairs_df['distances'] < distance_cutoff]
    monomer_pairs_df[['ai_name','ai_resnum']] = monomer_pairs_df.ai.str.split("_", expand=True)
    monomer_pairs_df[['aj_name','aj_resnum']] = monomer_pairs_df.aj.str.split("_", expand=True)
    monomer_pairs_df = monomer_pairs_df.astype({"ai_resnum": int, "aj_resnum": int})
    monomer_pairs_df.drop(monomer_pairs_df[abs(monomer_pairs_df['aj_resnum'] - monomer_pairs_df['ai_resnum']) < distance_residue].index, inplace=True)

    return monomer_pairs_df    

In [10]:
random_coil = MDAnalysis.Universe(reference_random_coil_structure, reference_random_coil_trajectory)
plainMD = MDAnalysis.Universe(reference_plainMD_structure, reference_plainMD_trajectory)
peptides = plainMD.select_atoms('all')
print('Residues: ', plainMD.residues)
print('Atoms: ', len(peptides))


# Creating the plainMD combinations to count the pairs.
# The same combinations will be used for Random Coil simulations.
atomtypes = []
for atom in peptides:
    atp = str(atom.name) + '_' + str(atom.resnum)
    atomtypes.append(atp)

pairs_list = list(itertools.combinations(atomtypes, 2))
pairs_ai, pairs_aj = [], []
for n in range(0, len(pairs_list)):
    i = pairs_list[n][0]
    pairs_ai.append(i)
    j = pairs_list[n][1]
    pairs_aj.append(j)

print('Pairs list: ',len(pairs_list))
print('Number of frames: ', len(plainMD.trajectory))
plainMD_frames = len(plainMD.trajectory)
random_coil_frames = len(random_coil.trajectory)

Residues:  <ResidueGroup [<Residue ASP, 1>, <Residue ALA, 2>, <Residue GLU, 3>, ..., <Residue VAL, 40>, <Residue ILE, 41>, <Residue ALA, 42>]>
Atoms:  319
Pairs list:  50721
Number of frames:  7921


/home/emanuele/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/emanuele/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:216: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "


In [13]:
job_list = []
for frame_index in range(plainMD.trajectory.n_frames):
    job_list.append(dask.delayed(make_pairs(atomgroup=peptides, frame_index=frame_index, pairs_ai=pairs_ai, pairs_aj=pairs_aj)))

monomer_plainMD_df = pd.concat(dask.compute(job_list)[0])
monomer_plainMD_df

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

,ai,aj,ai_name,aj_name,ai_resnum,aj_resnum,distances
343,CA_1,CE1_4,CA,CE1,1,4,5.385586
658,CB_1,CD1_4,CB,CD1,1,4,5.350300
659,CB_1,CE1_4,CB,CE1,1,4,4.225743
660,CB_1,CZ_4,CB,CZ,1,4,4.633304
974,CG_1,CE1_4,CG,CE1,1,4,5.045256
...,...,...,...,...,...,...,...
50611,C_40,CA_42,C,CA,40,42,4.963446
50612,C_40,CB_42,C,CB,40,42,4.883984
50624,O_40,N_42,O,N,40,42,3.438983
50625,O_40,CA_42,O,CA,40,42,4.350930


In [14]:
job_list = []
for frame_index in range(random_coil.trajectory.n_frames):
    job_list.append(dask.delayed(make_pairs(atomgroup=peptides, frame_index=frame_index, pairs_ai=pairs_ai, pairs_aj=pairs_aj)))

monomer_random_coil_df = pd.concat(dask.compute(job_list)[0])
monomer_random_coil_df

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

,ai,aj,ai_name,aj_name,ai_resnum,aj_resnum,distances
343,CA_1,CE1_4,CA,CE1,1,4,5.385586
658,CB_1,CD1_4,CB,CD1,1,4,5.350300
659,CB_1,CE1_4,CB,CE1,1,4,4.225743
660,CB_1,CZ_4,CB,CZ,1,4,4.633304
974,CG_1,CE1_4,CG,CE1,1,4,5.045256
...,...,...,...,...,...,...,...
50611,C_40,CA_42,C,CA,40,42,5.110741
50612,C_40,CB_42,C,CB,40,42,4.986042
50624,O_40,N_42,O,N,40,42,3.884254
50625,O_40,CA_42,O,CA,40,42,4.907282


In [ ]:
import multiprocessing as mp
from multiprocessing import Queue, Pool, Process

total_pairs = len(pairs_list)
# count_ai, count_aj, count_plainMD_distance, count_plainMD_ratio, count_random_coil_ratio, average_plainMD_distance = [], [], [], [], [], []


def dist_count(q):
    count_ai, count_aj, count_plainMD_distance, count_plainMD_ratio, count_random_coil_ratio, average_plainMD_distance = [], [], [], [], [], []
    for n_counter, pair in enumerate(pairs_list, 1):
        print(f'Processing {n_counter} out of {total_pairs}: {pair}')
        # filtering the data frame based on the pairs values
        count_ai.append(pair[0])
        count_aj.append(pair[1])
        # salvati il df che serve per la media delle distanze e del sigma
        counts_plainMD_df = monomer_plainMD_df[(monomer_plainMD_df['ai'] == pair[0]) & (monomer_plainMD_df['aj'] == pair[1])]
        counts_random_coil_df = monomer_random_coil_df[(monomer_random_coil_df['ai'] == pair[0]) & (monomer_random_coil_df['aj'] == pair[1])]

        average_plainMD_distance.append(counts_plainMD_df['distances'].mean())
        count_plainMD_distance.append(len(counts_plainMD_df))
        count_plainMD_ratio.append(len(counts_plainMD_df)/plainMD_frames)

        count_random_coil_ratio.append(len(counts_random_coil_df)/random_coil_frames)

q = Queue()
# process_list = [ Process(target=dist_count, args=(q, pairs_list, )) for _ in range(20) ]

c_size = 
split_list = [ pairs_list[x:x+c_size] for x in range(0, pairs_list, c_size)]

for i in range(20):


[ p.start() for p in process_list ]
[ p.join() for p in process_list ]


In [20]:
total_pairs

50721

In [ ]:
count_ai, count_aj, count_plainMD_distance, count_plainMD_ratio, count_random_coil_ratio, average_plainMD_distance = [], [], [], [], [], []

total_pairs = len(pairs_list)
for n_counter, pair in enumerate(pairs_list, 1):
    print(f'Processing {n_counter} out of {total_pairs}: {pair}')
    # filtering the data frame based on the pairs values
    count_ai.append(pair[0])
    count_aj.append(pair[1])
    # salvati il df che serve per la media delle distanze e del sigma
    counts_plainMD_df = monomer_plainMD_df[(monomer_plainMD_df['ai'] == pair[0]) & (monomer_plainMD_df['aj'] == pair[1])]
    counts_random_coil_df = monomer_random_coil_df[(monomer_random_coil_df['ai'] == pair[0]) & (monomer_random_coil_df['aj'] == pair[1])]

    average_plainMD_distance.append(counts_plainMD_df['distances'].mean())
    count_plainMD_distance.append(len(counts_plainMD_df))
    count_plainMD_ratio.append(len(counts_plainMD_df)/plainMD_frames)

    count_random_coil_ratio.append(len(counts_random_coil_df)/random_coil_frames)


In [7]:
pairs_count = pd.DataFrame(columns=['ai', 'aj', 'count', 'ratio', 'average_distance'])
pairs_count['ai'] = count_ai
pairs_count['aj'] = count_aj
pairs_count['count'] = count_plainMD_distance
pairs_count['ratio'] = count_plainMD_ratio
pairs_count['average_distance'] = average_plainMD_distance
pairs_count['random_coil_treshold'] = count_random_coil_ratio
pairs_count.sort_values(by = ['ratio'], inplace = True, ascending=False)
pairs_count.dropna()
pairs_count


,ai,aj,count,ratio,average_distance
1899,C_1,N_3,7921,1.0,4.007297
26422,C_12,N_14,7921,1.0,3.578960
43225,O_23,N_25,7921,1.0,3.601102
6182,C_3,N_5,7921,1.0,3.874453
30631,C_14,N_16,7921,1.0,3.970003
...,...,...,...,...,...
36190,CB_18,N_19,0,0.0,NaN
36189,CB_18,O_18,0,0.0,NaN
36188,CB_18,C_18,0,0.0,NaN
36187,CB_18,CG2_18,0,0.0,NaN


In [8]:
file = open('monomer_pairs_parallel_check.txt', 'w')
file.write(pairs_count.to_string(index=False, header=False))
file.close()

In [ ]:
def pairs_counter(pair, monomer_pairs_df, total_frames):
    count_distance, count_ratio, average_distance = [], [], []
    #print(f'Processing {n_counter} out of {total_pairs}: {pair}')
    # filtering the data frame based on the pairs values
    # salvati il df che serve per la media delle distanze e del sigma
    counts_df = monomer_pairs_df[(monomer_pairs_df['ai'] == pair[0]) & (monomer_pairs_df['aj'] == pair[1])]
    average_distance.append(counts_df['distances'].mean())
    count_distance.append(len(counts_df))
    count_ratio.append(len(counts_df)/total_frames)

    return (pair[0], pair[1], average_distance, count_distance, count_ratio)


In [ ]:
import ipyparallel as ipp
ippc = ipp.Client()
ippc.ids

In [ ]:
from ipyparallel import Client

rc = Client()
view = rc.load_balanced_view()


count_ai, count_aj, async_results = [], [], []
for pair in pairs_list[:100]:
    count_ai.append(pair[0])
    count_aj.append(pair[1])
    async_result = view.apply_async(pairs_counter, pair, monomer_pairs_df, total_frames)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]
print(results)

In [ ]:
job_list = []
count_ai, count_aj, count_distance, count_ratio, average_distance = [], [], [], [], []

for pair in pairs_list:
    n_counter = 1
    print(f'Processing {n_counter} out of {total_pairs}: {pair}')
    job_list.append(dask.delayed(pairs_counter(pair=pair, monomer_pairs_df=monomer_pairs_df)))
    n_counter += 1
    
pairs_count = pd.concat(dask.compute(job_list)[0])
pairs_count